In [ ]:
import requests
from dotenv import load_dotenv
import os

# 환경변수 불러오기
load_dotenv()

# 환경변수에서 키, 엔드포인트 불러오기
endpoint_base = os.getenv("clu_endpoint")  
api_key = os.getenv("clu_key")
api_id = os.getenv("clu_id")
project_name = "PPCLU"
deployment_name = "ppv1-test"


# 요청 헤더
headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    "Content-Type": "application/json",
    "Apim-Request-Id" : api_id
}

# 요청 본문 (Query)
query = {
    "kind": "Conversation",
    "analysisInput": {
        "conversationItem": {
            "id": "1",
            "participantId": "user",
            "text": "예산은 3천만 원 정도 있고, 서울에서 3월 말쯤 뮤지컬 하나 해보고 싶어요",
            "modality": "text",
            "language": "ko"
        }
    },
    "parameters": {
        "projectName": project_name,
        "deploymentName": deployment_name,
        "verbose": True,
        "stringIndexType": "TextElement_V8"
    }
}

# 응답 보기 좋게 출력하는 함수
def pretty_print_clu_result(clu_response):
    try:
        result = clu_response['result']
        prediction = result['prediction']
        intent = prediction['topIntent']
        entities = prediction.get('entities', [])

        print(f'당신: {query["analysisInput"]["conversationItem"]["text"]}')
        print(" 추출된 엔터티:")
        for ent in entities:
            category = ent['category']
            text = ent['text']
            score = ent.get("confidenceScore", 0)
            print(f"- {category}: {text} (신뢰도: {score:.2f})")
    except Exception as e:
        print(" 오류 발생:", e)

# API 호출
response = requests.post(
    endpoint_base,
    headers=headers,
    json=query
)

# 결과 저장
if response.status_code == 200:
    clu_response = response.json()  #  저장
    pretty_print_clu_result(clu_response)  # 출력도 계속 하고
else:
    print(f" 요청 실패: {response.status_code}")
    print(response.text)
    clu_response = None  # 실패 시에도 변수 정의

# 클루 결과 객체를 그대로 다음 파이프라인으로 넘기기 위해 출력
clu_response  # Jupyter Notebook에서 마지막 줄에 출력



In [19]:
import os
import json
import requests
import time
from openai import AzureOpenAI
from dotenv import load_dotenv
from openai.types.beta.threads import Message

load_dotenv()

def print_pretty_messages(messages):
    print("\n📨 메시지 대화 로그\n" + "-"*40)
    for message in reversed(messages.data):  # 최신순 → 사용자 → GPT 순서로 정렬
        role = "🧑 사용자" if message.role == "user" else "🤖 GPT"
        print(f"\n[{role}]")
        for block in message.content:
            if block.type == "text":
                print(block.text.value)
        print("-"*40)

client = AzureOpenAI(
  azure_endpoint = os.getenv("oai_assistant_endpoint"),
  api_key= os.getenv("oai_assistant_key"),
  api_version="2024-05-01-preview"
)

assistant = client.beta.assistants.create(
  model="gpt-4o", # replace with model deployment name.
  instructions="""너는 한국의 공연 정보를 잘 알고 있는 똑똑한 문화예술 전문가 도우미야.
사용자가 질문하면, 벡터 저장소에서 관련 정보를 찾아서 친절하고 정확하게 알려줘.
응답은 정중한 말투를 사용하며, 공연/ 예매정보/상세정보/ 통계등을 포함해 요약해줘.""",
  tools=[{"type":"file_search"},{"type":"code_interpreter"}],
  tool_resources={"file_search":{"vector_store_ids":["vs_rk9EOjktnff6RYK12Hx5jn7J"]},"code_interpreter":{"file_ids":[]}},
  temperature=0.3,
  top_p=0.95
)

# Create a thread
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=input("내용을 입력하세요")
)


# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

if run.status == 'completed':
  messages = client.beta.threads.messages.list(thread_id=thread.id)
  print(print_pretty_messages(messages))
elif run.status == 'requires_action':
  # the assistant requires calling some functions
  # and submit the tool outputs back to the run
  pass
else:
  print(run.status)



📨 메시지 대화 로그
----------------------------------------

[🧑 사용자]
25년 1월 공연정보
----------------------------------------

[🤖 GPT]
2025년 1월에 진행된 공연 정보는 다음과 같습니다:

1. **뮤지컬**
   - **전설의 놀이왕: 나는 끝까지 놀다갈거야!**
     - 장소: 관악아트홀 (서울특별시)
     - 일정: 2025.01.25【4:0†source】

   - **오셀로의 재심**
     - 장소: 스케치홀 (서울특별시)
     - 일정: 2025.01.08 ~ 2025.01.26【4:19†source】

2. **대중음악**
   - **한밤의 유U;콘서트 (1월)**
     - 장소: (재) 영화의전당 (부산광역시)
     - 일정: 2025.01.15【4:0†source】

   - **주말재즈클럽 (1월)**
     - 장소: 파닥파닥클럽 (경기도)
     - 일정: 2025.01.03 ~ 2025.01.31【4:14†source】

3. **클래식 음악**
   - **Breeze Ensemble pour vents 신년음악회: Symphony of breeze**
     - 장소: 부천아트센터 (경기도)
     - 일정: 2025.01.18【4:14†source】

   - **콘사노 체임버오케스트라 신년음악회: La Danza**
     - 장소: 세종문화회관 (서울특별시)
     - 일정: 2025.01.25【4:0†source】

4. **국악**
   - **새해국악연**
     - 장소: 국립국악원 (서울특별시)
     - 일정: 2025.01.23【4:7†source】

   - **음악으로 물들다 [대구]**
     - 장소: 아트스페이스 루모스 (대구광역시)
     - 일정: 2025.01.20【4:1†source】

5. **특별 공연**
   - **설민석의 한국사 대모험: 영웅의 시간**
    